<a href="https://colab.research.google.com/github/marco-luzzara/boardgame-complexity-predictor/blob/master/extract_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
WORKING_LOCALLY = bool(os.getenv('WORKING_LOCALLY'))

if WORKING_LOCALLY:
    DATASET_FILE_PATH = 'data/dataset.csv'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    DATASET_FILE_PATH = '/content/drive/My Drive/Projects/IRBoardGameComplexity/dataset.csv'
    !pip install fastcoref==2.0.*

In [2]:
from fastcoref import spacy_component
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
# nlp.add_pipe("fastcoref", 
#              config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref'})
nlp.add_pipe("fastcoref")

Downloading:   0%|          | 0.00/819 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/362M [00:00<?, ?B/s]

In [3]:
import logging

logger = logging.getLogger('bgg_predict')
logger.handlers.clear()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.debug('test')

In [16]:
import re
from typing import List
from dataclasses import dataclass

regex_mail = re.compile(r'\w+(?:\.\w+)*?@\w+(?:\.\w+)+')
regex_sentence_too_short = re.compile(r'\..{1,5}(?=\.)')

@dataclass
class Sentence:
    content: str
    start: int
    end: int
    
    def does_include_pos(self, pos: int) -> bool:
        return self.start <= pos <= self.end

def clean_text(text: str) -> str:
    for clean_function in [lambda x: regex_mail.sub('', x),
                           lambda x: regex_sentence_too_short.sub('', x)]:
        text = clean_function(text)
    return text

def get_sentences_from_text(text: str) -> List[Sentence]:
    # assert no continuous dots because of text cleared while building the dataset
    sentences = text.split('.')
    
    res = []
    char_accumulator = 0
    for sentence in sentences:
        res.append(Sentence(sentence, char_accumulator, char_accumulator + len(sentence) - 1))
        # + 1 because of the dot at the end of the sentence
        char_accumulator += len(sentence) + 1
        
    if res[-1].content == '':
        res.pop()
        
    return res

test_text = 'this is a test (me@gmail.it) that will be cleaned. with. only 2 sentences.'
test_text = clean_text(test_text)
test_sentences = get_sentences_from_text(test_text)
assert len(test_sentences) == 2
test_sentences

[Sentence(content='this is a test () that will be cleaned', start=0, end=37),
 Sentence(content=' only 2 sentences', start=39, end=55)]

In [17]:
def get_sentences_from_clusters(clusters: List[List[Tuple[int, int]]], sentences: List[Sentence]) -> List[List[int]]:
    # + sentences[0] because sentences are built from the entire text and not from the current group
    clusters_on_sentences = [[next(filter(lambda x: x[1].start <= entity[0] + sentences[0].start <= x[1].end, enumerate(sentences)))[0] 
                              for entity in cluster]
                             for cluster in clusters]

    return clusters_on_sentences

text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'
sentences = get_sentences_from_text(text)
clusters = [[(0, 5), (39, 42), (79, 82)]]
get_sentences_from_clusters(clusters, sentences)

[[0, 1, 1]]

In [18]:
from typing import List, Set
from itertools import groupby
from operator import itemgetter
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

def get_rule_groups_from_sentence_clusters(sentences: List[Sentence], sentence_clusters: List[List[int]]) -> List[List[int]]:
    def normalize_group(group: Set[int]) -> List[List[int]]:
        '''each group could contain multiple consecutive sublists. this method split these sublists'''
        res = []

        # https://stackoverflow.com/a/23861347/5587393
        for k, g in groupby(enumerate(sorted(list(group))), lambda x: x[0] - x[1]):
            res.append(list(map(itemgetter(1), g)))

        return res
    # the graph is built as a directed sparse graph where the first element of each cluster
    # is connected to the other elements in the same cluster
    graph = [[0 for _ in range(len(sentences))] for __ in range(len(sentences))]
    for cluster in sentence_clusters:
        for sentence in cluster[1:]:
            graph[cluster[0]][sentence] = 1

    # find the connected components of the graph created from the clusters returned after coref     
    graph = csr_matrix(graph)
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    groups = [set() for _ in range(n_components)]
    for i, label in enumerate(labels):
        groups[label].add(i)

    return [norm_group for group in groups for norm_group in normalize_group(group)]

In [ ]:
from typing import List, Tuple
import pandas as pd
import ast

def get_rules(text: str) -> List[str]:
    text = clean_text(text)
    sentences = get_sentences_from_text(text)
    
    GROUP_STEP_OFFSET = 2
    # I create groups of 4 sentences to speed up the process of finding connected sentences
    # and to make sure to find connected sentences not immediately adjacent
    sentences_groups = [sentences[i:min(i+4, len(sentences))] for i in range(0, len(sentences) - 2, GROUP_STEP_OFFSET)]

    cluster_groups = []
    for i, group in enumerate(sentences_groups):
        group_text = nlp('.'.join(map(lambda s: s.content, group)))
        group_coref_clusters = group_text._.coref_clusters
        group_sentence_clusters = get_sentences_from_clusters(group_coref_clusters, group)
        # + i * GROUP_STEP_OFFSET to retrieve the actual index of the sentence
        cluster_groups.extend([sentence_id + i * GROUP_STEP_OFFSET for sentence_id in gsc] 
                               for gsc in group_sentence_clusters)
        
    rule_groups = get_rule_groups_from_sentence_clusters(sentences, cluster_groups)                                                                                
    
    return ['. '.join([sentences[s_index].content for s_index in group]) for group in rule_groups]

with pd.read_csv(DATASET_FILE_PATH, chunksize=1, converters={ 'family': ast.literal_eval }) as reader:
    df = next(reader)
    doc = df['rulebook'].iloc[0]
    rules = get_rules(doc)

rules

In [ ]:
from typing import Tuple
import pandas as pd
import ast
from IPython.display import clear_output

def get_rules_features(text: str) -> Tuple[int, float]:
    rules = get_rules(text)
    rule_count = len(rules)
    return rule_count, len(text) / rule_count

def remove_columns_prefix(df) -> None:
    '''remove prefix 'info.' from the columns of df'''
    df.rename(columns=lambda c: c.rsplit('.', 1)[-1], inplace=True)
    
df_features = pd.DataFrame()
# ast.literal_eval converts the family column string into a python array
with pd.read_csv(DATASET_FILE_PATH, chunksize=5, converters={ 'family': ast.literal_eval }) as reader:
    for df in reader:
        remove_columns_prefix(df)
        df_rules_features = df.apply(lambda x: pd.Series(get_rules_features(x.rulebook), 
                                     index=['rule_count', 'avg_rule_len']), axis='columns')
        df_features = pd.concat([df_features, df[['averageweight', 'playingtime', 'family']].join(df_rules_features)])
        clear_output(wait=True)
        display(df_features.tail())
        
# one-hot encoding "family" field 
# from https://stackoverflow.com/questions/71401193/one-hot-encoding-in-python-for-array-values-in-a-dataframe
df_features = df_features.join(df_features.pop('family').apply('|'.join).str.get_dummies())
df_features.head()

PROCESSED_DATASET_FILE_PATH = 'data/processed_dataset.csv' if WORKING_LOCALLY \
    else '/content/drive/My Drive/Projects/IRBoardGameComplexity/processed_dataset.csv'

df_features.to_csv(PROCESSED_DATASET_FILE_PATH, header=True, index=False, mode='w')    
if not WORKING_LOCALLY:
    drive.flush_and_unmount()

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]